In [1]:
## Metaclasses

In [2]:
class A:
    pass

In [3]:
type(A)

type

In [4]:
class Meta(type):
    pass

class MyClass(metaclass=Meta):
    pass

class MySubclass(MyClass):
    pass

In [5]:
type(MySubclass)

__main__.Meta

In [7]:
C = type("C", (), {})

In [8]:
type(C)

type

In [14]:
class DocsRequired(type):
    def __new__(cls, name, bases, dct):
        inst = super().__new__(cls, name, bases, dct)
        if not inst.__doc__:
            raise RuntimeError("documentation required")
        return inst

In [16]:
class A(metaclass=DocsRequired):
    """
    Some docs.
    """
    pass

In [17]:
## Descriptors

In [18]:
class A:
    v = 123

In [19]:
A.__dict__

mappingproxy({'__module__': '__main__',
              'v': 123,
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None})

In [20]:
a = A()

In [22]:
a.__dict__

{}

In [24]:
a.__class__.__dict__

mappingproxy({'__module__': '__main__',
              'v': 123,
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None})

Descriptors:

* `__get__`, `__set__`, `__delete__`

In [25]:
class A:
    x = 10

In [26]:
a = A()

In [27]:
a.x

10

In [28]:
class D:
    def __get__(self, obj, objtype=None):
        print("D.__get__", obj)
        return 10

In [29]:
class A:
    x = D()

In [30]:
a = A()

In [31]:
a.x

D.__get__ <__main__.A object at 0x73a3bf6c4740>


10

In [32]:
import os

In [41]:
class DirectorySize:
    def __get__(self, obj, objtype=None):
        return len(os.listdir(obj.dirname))

In [42]:
class Directory:

    size = DirectorySize()

    def __init__(self, dirname):
        self.dirname = dirname

In [43]:
d = Directory(".")

In [44]:
d.size

3

In [70]:
class IntRange:

    def __init__(self, min=None, max=None):
        self.min = min
        self.max = max
    
    def __set_name__(self, owner, name):
        self.private_name = "_" + name
    
    def __get__(self, obj, objtype):
        return getattr(obj, self.private_name)
    
    def __set__(self, obj, value):
        if self.min is not None:
            if value < self.min:
                raise ValueError("value too small")
        if self.max is not None:
            if value > self.max:
                raise ValueError("value too large")
        setattr(obj, self.private_name, value)
    


In [71]:
class Record:
    x = IntRange(min=10, max=20)
    y = IntRange(min=0)

In [72]:
record = Record()

In [73]:
record.__dict__

{}

In [74]:
record.x = 10

In [75]:
from abc import ABC, abstractmethod

class Validator(ABC):

    def __set_name__(self, owner, name):
        self.private_name = '_' + name

    def __get__(self, obj, objtype=None):
        return getattr(obj, self.private_name)

    def __set__(self, obj, value):
        self.validate(value)
        setattr(obj, self.private_name, value)

    @abstractmethod
    def validate(self, value):
        pass

class OneOf(Validator):

    def __init__(self, *options):
        self.options = set(options)

    def validate(self, value):
        if value not in self.options:
            raise ValueError(f'Expected {value!r} to be one of {self.options!r}')

class Number(Validator):

    def __init__(self, minvalue=None, maxvalue=None):
        self.minvalue = minvalue
        self.maxvalue = maxvalue

    def validate(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError(f'Expected {value!r} to be an int or float')
        if self.minvalue is not None and value < self.minvalue:
            raise ValueError(
                f'Expected {value!r} to be at least {self.minvalue!r}'
            )
        if self.maxvalue is not None and value > self.maxvalue:
            raise ValueError(
                f'Expected {value!r} to be no more than {self.maxvalue!r}'
            )
        
class String(Validator):

    def __init__(self, minsize=None, maxsize=None, predicate=None):
        self.minsize = minsize
        self.maxsize = maxsize
        self.predicate = predicate

    def validate(self, value):
        if not isinstance(value, str):
            raise TypeError(f'Expected {value!r} to be an str')
        if self.minsize is not None and len(value) < self.minsize:
            raise ValueError(
                f'Expected {value!r} to be no smaller than {self.minsize!r}'
            )
        if self.maxsize is not None and len(value) > self.maxsize:
            raise ValueError(
                f'Expected {value!r} to be no bigger than {self.maxsize!r}'
            )
        if self.predicate is not None and not self.predicate(value):
            raise ValueError(
                f'Expected {self.predicate} to be true for {value!r}'
            )

In [76]:
class Component:

    name = String(minsize=3, maxsize=10, predicate=str.isupper)
    kind = OneOf('wood', 'metal', 'plastic')
    quantity = Number(minvalue=0)

    def __init__(self, name, kind, quantity):
        self.name = name
        self.kind = kind
        self.quantity = quantity

In [80]:
c = Component("DESK", "wood", 1)

In [81]:
c

## Iterator, Generator

In [82]:
for x in [1, 2, 3]:
    print(x)

1
2
3


In [85]:
for i in range(10):
    print(i)

0
1
2
3
4
5
6
7
8
9


In [86]:
[i * i for i in range(10)]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [97]:
g = (i * i for i in range(10))

In [99]:
for i in g:
    print(i)

0
1
4
9
16
25
36
49
64
81


In [117]:
g = os.walk("/home/tir")

In [118]:
len(g)

TypeError: object of type 'generator' has no len()

In [131]:
class Gen:

    def __iter__(self):
        return self

    def __next__(self):
        return 42
    

In [133]:
g = Gen()

In [134]:
next(g)

42

In [135]:
import itertools

In [136]:
for i in itertools.islice(g, 3):
    print(i)

42
42
42


In [143]:
class Gen:

    def __init__(self):
        self.i = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        self.i += 1
        if self.i < 4:
            return self.i
        raise StopIteration

In [145]:
g = Gen()
for i in g:
    print(i)

1
2
3


In [139]:
next(g)

1

In [140]:
next(g)

2

In [141]:
next(g)

3

In [146]:
# yield

In [147]:
class Gen:

    def __init__(self):
        self.i = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        self.i += 1
        if self.i < 4:
            return self.i
        raise StopIteration

In [155]:
def gen():
    while True:
        yield 42

In [156]:
gen()

<generator object gen at 0x73a3be6562a0>

In [157]:
g = gen()

In [160]:
for i in itertools.islice(g, 1):
    print(i)

42


In [161]:
def gen():
    i = 0
    while i < 4:
        i += 1
        yield i


In [162]:
g = gen()

In [163]:
for i in g:
    print(i)

1
2
3
4
